In [1]:
from dotenv import load_dotenv
load_dotenv()

import os
import asyncio
from postchain_client_py import BlockchainClient
from postchain_client_py.blockchain_client.types import NetworkSettings, Operation, Transaction
    
# Initialize network settings
settings = NetworkSettings(
    node_url_pool=["http://localhost:7740"],
    blockchain_iid=0,
)
    
# Create client and execute query
client = await BlockchainClient.create(settings)

In [2]:
sensor_result = await client.query("get_sensor_count")
telemetry_result = await client.query("get_telemetry_count")
print(f"Sensor Count: {sensor_result}")
print(f"Telemetry Count: {telemetry_result}")


Sensor Count: 0
Telemetry Count: 0


In [3]:
from coincurve import PrivateKey

private_bytes = bytes.fromhex(os.getenv("PRIV_KEY"))
private_key = PrivateKey(private_bytes)
public_key = private_key.public_key.format(compressed=True)
print(public_key.hex())

async def sign_and_send_transaction(operation):
    transaction = Transaction(
        operations=[operation],
        signers=[public_key],
        signatures=None,
        blockchain_rid=client.config.blockchain_rid
    )
    signed_tx = await client.sign_transaction(transaction, private_bytes)
    receipt = await client.send_transaction(signed_tx, do_status_polling=True)
    return receipt


03922b871a76a243957bdc26f1204c7612f221c9213697afef17a9ac602b84a042


In [4]:
import json
# Create sensor
metadata = json.dumps({"robot": "demo", "status": "active"})

operation = Operation(
    op_name="create_sensor",
    args=[
        "ChromBot",
        metadata
    ]
)
receipt = await sign_and_send_transaction(operation)
print(receipt)
sensor_result = await client.query("get_sensor_count")
print(f"Sensor Count: {sensor_result}")

TransactionReceipt(status='confirmed', status_code=200, transaction_rid=b'ox3\xf7bu]\x93\x7f\x08Ix\x9b\xa2 \x93\x18\xbc\x13\xb6\xf1/\xf6\x1f=\n\x19#\n\xfb\x7f\xf8', message=None, cluster_anchored_tx=None, system_anchored_tx=None)
Sensor Count: 1


In [5]:
operation = Operation(
    op_name="add_data",
    args=[
        "ChromBot",
        "69"
    ]
)

print(operation)

Operation(op_name='add_data', args=['ChromBot', '69'])


In [6]:
transaction = Transaction(
    operations=[operation],
    signers=[public_key],
    signatures=None,
    blockchain_rid=client.config.blockchain_rid
)
print(transaction)

Transaction(operations=[Operation(op_name='add_data', args=['ChromBot', '69'])], signers=[b'\x03\x92+\x87\x1av\xa2C\x95{\xdc&\xf1 Lv\x12\xf2!\xc9!6\x97\xaf\xef\x17\xa9\xac`+\x84\xa0B'], signatures=None, blockchain_rid='E880B57E6C5EA1A1DA59D7E7C53382951A6E4AD5E8B7081BC17EA5E7C0D424B1')


In [7]:
signed_tx = await client.sign_transaction(transaction, private_bytes)
receipt = await client.send_transaction(signed_tx, do_status_polling=True)

print(receipt)   

TransactionReceipt(status='confirmed', status_code=200, transaction_rid=b'H1\x9fX\x08\x8b\xdd\x9f4\xca\xef@\xa1J\xff\x03\xf2_\xcd\xbc\x88~\x19>#\xec\xf6\xf8\xecid\xb7', message=None, cluster_anchored_tx=None, system_anchored_tx=None)


In [10]:
result = await client.query("get_all_data")
print(f"Telemetry Data ({len(result)} records):")

for i, record in enumerate(result, 1):
    sensor_id = record[0]
    sensor_name = record[1]
    metadata = record[2]
    creator = record[3]
    battery = record[4]
    timestamp = record[5]
    print(f"  {i}. Sensor: {sensor_name}, ID: {sensor_id}, Metadata: {metadata}, Battery: {battery}%, Timestamp: {timestamp} Creator: {creator.hex()}")

Telemetry Data (1 records):
  1. Sensor: ChromBot, ID: 1758789186414, Metadata: {"robot":"demo","status":"active"}, Battery: 69%, Timestamp: 1758789191652 Creator: 03922b871a76a243957bdc26f1204c7612f221c9213697afef17a9ac602b84a042


In [9]:
# Clean up
await client.rest_client.close()